<a href="https://colab.research.google.com/github/TonmoyTalukder/Rank-Your-Summaries-Enhancing-Bengali-Text-Summarization-via-Ranking-based-Approach/blob/main/Summary%20Generation%20with%20Pre-trained%20Models/mT5_m2m_crossSum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Loading

## Dataset I

In [ ]:
#install dataset
!pip install datasets

In [ ]:
from datasets import load_dataset
from pprint import pprint

In [ ]:
dataset = load_dataset('csebuetnlp/xlsum', 'bengali', split='train')

In [ ]:
print("Dataset length: ", len(dataset))
print("\n First item dataset[0]: ")
pprint(dataset[1])

In [ ]:
pprint(len(dataset[0]['text']))
pprint(type(dataset[0]['text']))

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)

df.head()

In [ ]:
df.head(10)

In [ ]:
df['text'][2]

In [ ]:
rows = df.shape[0]
rows

8102

## Dataset II

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/SCDataset/Final/text_summarization.csv')

df.head()

,category,summary,text
0,technology,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,bangladesh,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,bangladesh,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,bangladesh,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,bangladesh,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...


In [ ]:
rows = df.shape[0]
rows

80331

# Install and Imports

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer


from transformers import XLMProphetNetForConditionalGeneration, XLMProphetNetTokenizer


from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer

In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Functions

In [ ]:
def cleanText(text2clean):
    empty_string = ""

    space = chr(32)
    flag  = 0

    for i in range(len(text2clean)):
        if text2clean[i] != "'" and text2clean[i] != "\n":
            if text2clean[i] == space and flag == 0:
                flag = 1
                empty_string = empty_string + text2clean[i]
            elif text2clean[i] != space:
                empty_string = empty_string + text2clean[i]
                flag = 0
    return empty_string

# Model Loading 7: csebuetnlp/mT5_m2m_crossSum

### Model Setup

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

# article_text = """Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down, the company said.  The policy includes the termination of accounts of anti-vaccine influencers.  Tech giants have been criticised for not doing more to counter false health information on their sites.  In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""
# article_text = normalize(df['text'][2])

model_name = "csebuetnlp/mT5_m2m_crossSum"
tokenizer7 = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model7 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

get_lang_id = lambda lang: tokenizer7._convert_token_to_id(
    model7.config.task_specific_params["langid_map"][lang][1]
)

target_lang = "bengali" # for a list of available language names see below

model7.to(device) # Add This Line

### Run dataset over the Model

In [ ]:
# Create empty DataFrame
genDf = pd.DataFrame(columns = ["title", "text", "givenSummary", "mT5_m2m_crossSum"])

# Load over the dataset
for i in range(rows):
    # if i < 1000:
    # if i > 999 and i < 2000:
    #if i > 1999 and i < 3000:
    # if i > 2999 and i < 4000:
    # if i > 3999 and i < 5000:
    #  if i > 4999 and i < 6000:
    # if i > 5999 and i < 7000:
    # if i > 6999 and i < 8000:
    if i > 7999:
        article_text = normalize(df['text'][i])
        # article_text = df['text'][0]

        input_ids = tokenizer7(
            # [WHITESPACE_HANDLER(article_text)],
            article_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"]

        output_ids = model7.generate(
            input_ids=input_ids.cuda(),
            decoder_start_token_id=get_lang_id(target_lang),
            max_length=400,
            min_length=64,
            no_repeat_ngram_size=2,
            num_beams=4,
        )[0]

        summary = tokenizer7.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        genDf.loc[i] = [df['title'][i], df['text'][i], df['summary'][i], summary]
        print(i, '  ', summary)
        #if i == 10:
           # break

# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v1000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v2000.csv')
#genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v3000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v4000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v5000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v6000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v7000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v8000.csv')
genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v8KPlus.csv')

## Concat Splitted Dataset

In [ ]:
df0 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v1000.csv")
df1 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v2000.csv")
df2 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v3000.csv")
df3 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v4000.csv")
df4 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v5000.csv")
df5 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v6000.csv")
df6 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v7000.csv")
df7 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v8000.csv")
df8 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum/mT5_m2m_crossSum_v8KPlus.csv")


frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8]
genDfCombined = pd.concat(frames)
genDfCombined.to_csv('/content/drive/MyDrive/SCDataset/mT5_m2m_crossSum_Combined_v.csv')